In [ ]:
import torch
#Check if you can use a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device : {device}")

In [ ]:
import geopandas as gpd
import os
from config import (
    FILE_PATH_TRAIN_POLYGONS,
    FILE_PATH_TEST_POLYGONS,
    DIR_TRAIN_ORTHOPHOTOS,
    DIR_TEST_ORTHOPHOTOS,
)

# === Chargement des polygones
gdf_train = gpd.read_file(FILE_PATH_TRAIN_POLYGONS)
gdf_test  = gpd.read_file(FILE_PATH_TEST_POLYGONS)

# === Mise à jour des chemins vers les orthophotos
gdf_train["file_path"] = gdf_train["id"].apply(
    lambda id: os.path.join(DIR_TRAIN_ORTHOPHOTOS, f"{id}.tif")
)

gdf_test["file_path"] = gdf_test["id"].apply(
    lambda id: os.path.join(DIR_TEST_ORTHOPHOTOS, f"{id}.tif")
)



In [ ]:
from torch.utils.data import random_split, DataLoader,Subset
from torch.utils.data import Dataset
from torchvision import transforms
import torch
from torchvision.models import resnet18,resnet50,resnet34
from torch import nn
from torch.optim import Adam
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
from PIL import Image
import os


class ortho(Dataset):
    def __init__(self, df, transform=None, target_column="Imperviousness"):
        self.data = df.reset_index(drop=True)
        self.transform = transform
        self.target_column = target_column

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data["file_path"].iloc[idx]
        image = Image.open(img_path).convert("RGB")
        target = float(self.data[self.target_column].iloc[idx])

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(target, dtype=torch.float32)



transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
num_epochs = 15
learning_rate = 1e-4

train_dataset = ortho(gdf_train.reset_index(drop=True), transform=transform_train,target_column="Imperviousness")
test_dataset  = ortho(gdf_test.reset_index(drop=True),  transform=transform_test,target_column="Imperviousness")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=False)

# Choose the model and the number of layers to unfreeze
model = resnet18(weights="IMAGENET1K_V1")
for param in model.parameters():
    param.requires_grad = False
for param in model.layer3.parameters():
    param.requires_grad = True
for param in model.layer4.parameters():
    param.requires_grad = True
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
)

model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
criterion = nn.MSELoss()

# === Entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device).view(-1, 1)
        optimizer.zero_grad()
        preds = torch.clip(model(x), min=0, max=100)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs} - loss : {running_loss / len(train_loader):.4f}")
    

In [ ]:
model.eval()
predictions = []
targets = []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device).view(-1, 1)

        preds = torch.clip(model(x), min=0, max=100)

        predictions.append(preds.cpu())
        targets.append(y.cpu())

# Concaténer toutes les batches
predictions = torch.cat(predictions).squeeze().numpy()
targets     = torch.cat(targets).squeeze().numpy()

r2=r2_score(targets,predictions)
mae=mean_absolute_error(targets,predictions)
rmse=np.sqrt(mean_squared_error(targets,predictions))

print(f"=====Evaluation=====")
print(f"R2 : {r2:.2f}")
print(f"MAE : {mae:.1f}")
print(f"RMSE: {rmse:1f}")

In [ ]:
from config import MODELS_PATH
print(MODELS_PATH)

# Save model's weight
torch.save(model.state_dict(), os.path.join(MODELS_PATH,"resnet_weights.pt"))
